In [1]:
# # combine sessionmaker Session, with-statement & begin() to start session, handle exceptions, rollback, commit and close at once
# with Session.begin() as session:
#     session.add(object)
#     session.add_all([objects])
#     result = session.query(Project).filter(Project.name == 'Test').scalar() # scalar is good if only one match is possible due to database constraints

In [1]:
import settings # type: ignore
from camtrappy.db.testdb import initialize_new_project
from camtrappy.io import parsing

if not 'initialized' in locals():
    # Set data Dir & Data Project (file parser)
    DATA_DIR = settings.data_dir_animals
    DATA_PROJECT = parsing.ProjectParser(name='Test', projectfolder=None, datafolder=DATA_DIR, restrict_to=[])

    # Create new db project with parsed data
    initialize_new_project(DATA_PROJECT)
    initialized = True

home:      n:\Mitarbeiter\Florian\projects\CamTrapPy
data:      n:\Mitarbeiter\Florian\projects\CamTrapPy\tests\data
notebooks: n:\Mitarbeiter\Florian\projects\CamTrapPy\notebooks
--Setup complete--
Parsing folder: 21_Herrenberg
Parsing folder: 25_Stockach
Parsing folder: 7_Ittlingen
2021-07-26 16:08:31,368 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-26 16:08:31,369 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("videos")
2021-07-26 16:08:31,370 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-26 16:08:31,373 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("videos")
2021-07-26 16:08:31,373 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-26 16:08:31,375 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("locations")
2021-07-26 16:08:31,376 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-26 16:08:31,379 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("locations")
2021-07-26 16:08:31,380 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-26 16:08:31,38

In [1]:
import settings # type: ignore
from camtrappy.db.testdb import Session
from camtrappy.db.schema import Video


with Session.begin() as session:
    q = session.query(Video)
r = q.limit(10).all()
r

home:      n:\Mitarbeiter\Florian\projects\CamTrapPy
data:      n:\Mitarbeiter\Florian\projects\CamTrapPy\tests\data
notebooks: n:\Mitarbeiter\Florian\projects\CamTrapPy\notebooks
--Setup complete--
2021-07-26 16:11:47,451 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-26 16:11:47,454 INFO sqlalchemy.engine.Engine SELECT videos.id AS videos_id, videos.path AS videos_path, videos.date AS videos_date, videos.time AS videos_time, videos.fps AS videos_fps, videos.duration AS videos_duration, videos.date_added AS videos_date_added, videos.location_id AS videos_location_id 
FROM videos
 LIMIT ? OFFSET ?
2021-07-26 16:11:47,455 INFO sqlalchemy.engine.Engine [generated in 0.00099s] (10, 0)


[Video(id=1, path=20180916_014502_3A26_00408CC583CD.mkv, date=2018-09-16, time=01:45:02, fps=None, duration=None),
 Video(id=2, path=20180916_025001_D8C1_00408CC583CD.mkv, date=2018-09-16, time=02:50:01, fps=None, duration=None),
 Video(id=3, path=20180916_025502_5F44_00408CC583CD.mkv, date=2018-09-16, time=02:55:02, fps=None, duration=None),
 Video(id=4, path=20180916_030002_1D11_00408CC583CD.mkv, date=2018-09-16, time=03:00:02, fps=None, duration=None),
 Video(id=5, path=20180916_033503_44B3_00408CC583CD.mkv, date=2018-09-16, time=03:35:03, fps=None, duration=None),
 Video(id=6, path=20181003_195526_2DEE.mkv, date=2018-10-03, time=19:55:26, fps=None, duration=None),
 Video(id=7, path=20181003_202031_28C4.mkv, date=2018-10-03, time=20:20:31, fps=None, duration=None),
 Video(id=8, path=20181003_205038_87B6.mkv, date=2018-10-03, time=20:50:38, fps=None, duration=None),
 Video(id=9, path=20181003_210541_4DB9.mkv, date=2018-10-03, time=21:05:41, fps=None, duration=None),
 Video(id=10, pat

In [2]:
from camtrappy.core.base import VideoList

vl = VideoList.from_dict([{'path': 'a/b/c', 'id': 1}, {'path': 'x/y/z', 'id':2}])

TypeError: __init__() keywords must be strings

In [1]:
import os
from pprint import pprint
from sqlalchemy import insert, select

import settings # type: ignore
from camtrappy.core.base import VideoLoader, VideoList, Video
from camtrappy.core.analysis import VideoAnalysis
from camtrappy.db.schema import Video, Location, Project
from camtrappy.db.testdb import initialize_new_project, Session
from camtrappy.io import parsing

import cv2

# Set data Dir & Data Project (file parser)
DATA_DIR = settings.data_dir_animals
DATA_PROJECT = parsing.Project(name='Test', projectfolder=None, datafolder=DATA_DIR, restrict_to=[])

# Create new db project with parsed data
initialize_new_project(DATA_PROJECT)

# See how the data is structured
pprint(DATA_PROJECT.data(full_path=True))
pprint(DATA_PROJECT.data(full_path=False))
pprint(DATA_PROJECT.data_flat())
print(DATA_PROJECT.locations())

home:      n:\Mitarbeiter\Florian\projects\CamTrapPy
data:      n:\Mitarbeiter\Florian\projects\CamTrapPy\tests\data
notebooks: n:\Mitarbeiter\Florian\projects\CamTrapPy\notebooks
--Setup complete--
Parsing location: 21_Herrenberg
Parsing location: 25_Stockach
Parsing location: 7_Ittlingen
2021-07-26 15:14:25,960 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-07-26 15:14:25,961 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("videos")
2021-07-26 15:14:25,962 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-26 15:14:25,965 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("videos")
2021-07-26 15:14:25,966 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-26 15:14:25,968 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("locations")
2021-07-26 15:14:25,969 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-26 15:14:25,970 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("locations")
2021-07-26 15:14:25,971 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-07-26 15:14

In [3]:
# Test VideoAnalysis
import cv2

os.chdir(DATA_DIR)
paths = {}
prev = None
for id, (loc, vid, _, _) in enumerate(DATA_PROJECT.data_flat()):
    if not prev: prev = loc
    if prev != loc: break
    paths[id] = {}
    paths[id]['path'] = f'{loc}\{vid}'

vl = VideoLoader.from_dict(paths)
vl.skip_frames=29
vl.start()

# loop over frames from the video file stream
while vl.more():
    video_id, frame = vl.read()
    # display the size of the queue on the frame
    cv2.putText(frame, f"Queue Size: {vl.Q.qsize()}",
        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    # display the video id on the frame
    cv2.putText(frame, f"Video ID: {video_id}",
        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    # show the frame
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vl.stop()

In [9]:
from dataclasses import dataclass, field
@dataclass(unsafe_hash=True)
class test:
    id: int
    path: str = field(default='path', compare=False)
    
    def test(self):
        return test(self.id)
    
t1 = test(1)
t2 = test(2)

d = {t1: 'a', t2: 'b'}
print(d)

t11 = t1.test()
t1 is t11

x = test(1, 'x')
y = test(2, 'y')

x.id = y
y.id = x

x.id.id.id.id.path

{test(id=1, path='path'): 'a', test(id=2, path='path'): 'b'}


'x'

In [1]:
import settings # type: ignore
from camtrappy.core.base import VideoLoader, VideoList, Video
from datetime import date, time
from collections import OrderedDict



v1 = Video('path1', date=date(2021, 7, 1), time=time(12, 31))
v11 = Video('path1')
v2 = Video('path2', date=date(2021, 7, 1), time=time(12, 30))
v3 = Video('path3', date=date(2021, 7, 1), time=time(12, 30))
l = [v1, v2, v3]
print(sorted(l))
print(sorted(l, reverse=True))

od = OrderedDict.fromkeys(sorted(l), 'hello')

print({v1: 'hello'}[v1], od[v1])

home:      n:\Mitarbeiter\Florian\projects\CamTrapPy
data:      n:\Mitarbeiter\Florian\projects\CamTrapPy\tests\data
notebooks: n:\Mitarbeiter\Florian\projects\CamTrapPy\notebooks
--Setup complete--
[Video(path='path2', id=None, date=datetime.date(2021, 7, 1), time=datetime.time(12, 30), fps=None, duration=None), Video(path='path3', id=None, date=datetime.date(2021, 7, 1), time=datetime.time(12, 30), fps=None, duration=None), Video(path='path1', id=None, date=datetime.date(2021, 7, 1), time=datetime.time(12, 31), fps=None, duration=None)]
[Video(path='path1', id=None, date=datetime.date(2021, 7, 1), time=datetime.time(12, 31), fps=None, duration=None), Video(path='path2', id=None, date=datetime.date(2021, 7, 1), time=datetime.time(12, 30), fps=None, duration=None), Video(path='path3', id=None, date=datetime.date(2021, 7, 1), time=datetime.time(12, 30), fps=None, duration=None)]
hello hello


In [5]:
vl = VideoList([Video('test')])
vl.chronological()

VideoList(videos=[Video(path='test', id=None, date=None, time=None, fps=None, duration=None)], idx=0)

In [ ]:
videos = [Video, Video, ...,]
objects = [Object, Object, ...,]

videos_objects = {Video: ObjectList}
objects_videos = {Object: VideoList}

class Object:
    
    id: int
    videos: OrderedDict[Video: Dict[frames=[], images=[]]]
    